# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = pd.read_csv("Output/CityWeatherData.csv")
data_map = pd.DataFrame(file)
data_map.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Rikitea,3,PF,1584331013,79,-23.12,-134.97,79.63,12.71
1,Fortuna,75,US,1584331126,80,40.60,-124.16,42.80,9.17
2,Hermanus,50,ZA,1584331037,77,-34.42,19.23,64.63,32.01
3,Severo-Kuril'sk,97,RU,1584331016,95,50.68,156.12,24.84,46.73
4,Port Keats,0,AU,1584331514,41,-14.25,129.55,94.44,3.98


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [4]:
# Generate map
fig1 = gmaps.figure(map_type="HYBRID")

# Add locations and weights
locations = data_map[["Lat", "Lng"]]
weights = data_map["Humidity"]

# Create heatmap layer
heatmap_layer = gmaps.heatmap_layer(
    locations=locations, weights=weights,
    max_intensity=max(data_map["Humidity"]), point_radius=5.0
)

# Add locations and weights to map
fig1.add_layer(heatmap_layer)
fig1

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Create dataframe to hold only locations with ideal weather conditions

# Filter out temps only between 70-80 degrees F (not inclusive)
ideal_weather_cities = data_map.loc[(data_map["Max Temp"] < 80) & (data_map["Max Temp"] > 70)]

# Remove cities with wind speed 10 mph or greater
ideal_weather_cities = ideal_weather_cities.loc[(ideal_weather_cities["Wind Speed"] < 10)]

# Remove cities with any cloudiness
ideal_weather_cities = ideal_weather_cities.loc[(ideal_weather_cities["Cloudiness"] == 0)]

ideal_weather_cities

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
59,Laï,0,TD,1584331537,13,9.40,16.30,70.02,4.94
215,Morondava,0,MG,1584331605,91,-20.28,44.28,71.62,0.65
238,Janakpur Dham,0,NP,1584331584,46,26.72,85.91,74.77,7.29
431,Iguape,0,BR,1584331656,98,-24.71,-47.56,71.62,1.97
461,São Francisco do Sul,0,BR,1584331663,81,-26.24,-48.64,75.00,1.16
509,Sharjah city,0,AE,1584331756,60,25.36,55.40,73.40,6.93


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = ideal_weather_cities

hotel_df["Hotel Name"] = ""
hotel_df = hotel_df.reset_index()
hotel_df

,index,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,59,Laï,0,TD,1584331537,13,9.40,16.30,70.02,4.94,
1,215,Morondava,0,MG,1584331605,91,-20.28,44.28,71.62,0.65,
2,238,Janakpur Dham,0,NP,1584331584,46,26.72,85.91,74.77,7.29,
3,431,Iguape,0,BR,1584331656,98,-24.71,-47.56,71.62,1.97,
4,461,São Francisco do Sul,0,BR,1584331663,81,-26.24,-48.64,75.00,1.16,
5,509,Sharjah city,0,AE,1584331756,60,25.36,55.40,73.40,6.93,


In [7]:
# Define parameters to search for hotel
target_search = "hotel"
radius = 5000

# Convert parameters to dictionary

params = {
    "keyword": target_search,
    "radius": radius,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [8]:
# Create list to hold markers
marker_list = []

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    # Set city name from city's row
    name = row["City"]
    
    # Get coordinates from city's data
    city_lat = row["Lat"]
    city_lng = row["Lng"]

    # Set coordinates for parameter to city's lat and lng
    coordinates = f"{city_lat}, {city_lng}"
    
    # Add city lat and lng to parameters
    params["location"] = coordinates
    
    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    
    # Set a variable to hold JSON results of API call
    hotel_data = response.json()
    
    try:
        # Set variables to hold info on Hotel's Name, Lat, and Long
        hotel_name = hotel_data["results"][0]["name"]
        hotel_city = row["City"]
        hotel_country = row["Country"]
        hotel_lat = hotel_data["results"][0]["geometry"]["location"]["lat"]
        hotel_lng = hotel_data["results"][0]["geometry"]["location"]["lng"]
        
        # If hotel not found within required radius, raise exception
    except IndexError:
        print(f"Hotel not found in the vicinity of {name}.")
        
        # Set data equal to empty
        hotel_name = "NaN"
        hotel_city = "NaN"
        hotel_country = "NaN"
        hotel_lat = "NaN"
        hotel_lng = "NaN"

    # Create dictionary to hold hotel data
    single_hotel_data = {
        "Hotel Name": hotel_name,
        "Hotel City": hotel_city,
        "Hotel Country": hotel_country,
        "Hotel Lat": hotel_lat,
        "Hotel Lng": hotel_lng
    }

    # Append dictionary to list of markers
    marker_list.append(single_hotel_data)

Hotel not found in the vicinity of Laï.


In [9]:
# use iterrows to iterate through pandas dataframe
for x in range (0, 10):
    try:
        # Insert Hotel Name into Hotel Name Column
        hotel_df.iloc[x, 10] = marker_list[x]["Hotel Name"]
    except IndexError:
        continue

In [10]:
# Clean dataframe and markers dictionary

# Clean dataframe
# Drop NaN from dataframe and reset index
hotel_df = hotel_df.loc[(hotel_df["Hotel Name"] != "NaN"), :]
hotel_df = hotel_df.reset_index()
hotel_df

,level_0,index,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,1,215,Morondava,0,MG,1584331605,91,-20.28,44.28,71.62,0.65,Palissandre Cote Ouest resort & SPA
1,2,238,Janakpur Dham,0,NP,1584331584,46,26.72,85.91,74.77,7.29,Hotel Welcome
2,3,431,Iguape,0,BR,1584331656,98,-24.71,-47.56,71.62,1.97,Holy Drop Skate House
3,4,461,São Francisco do Sul,0,BR,1584331663,81,-26.24,-48.64,75.00,1.16,Hotel VillaReal São Francisco do Sul
4,5,509,Sharjah city,0,AE,1584331756,60,25.36,55.40,73.40,6.93,Hilton Sharjah


In [11]:
# Clean markers

# Loop through all items in marker list
for x in range(len(marker_list)-1):
    try:
        # If hotel name in marker list is not available, delete
        if marker_list[x]["Hotel Name"] == "NaN":
            del marker_list[x]
    except IndexError:
        continue

In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

In [14]:
# Display Map
fig2 = gmaps.figure(map_type="HYBRID")

# Add locations and weights
locations = hotel_df[["Lat", "Lng"]]
weights = hotel_df["Humidity"]

# Create heatmap layer
heatmap_layer = gmaps.heatmap_layer(
    locations=locations, weights=weights,
    max_intensity=max(hotel_df["Humidity"]), point_radius=5.0
)

# Add locations and weights to map
fig2.add_layer(heatmap_layer)
fig2.add_layer(marker_layer)
fig2

Figure(layout=FigureLayout(height='420px'))